In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q bitsandbytes datasets accelerate
!pip install transformers
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to bui

In [ ]:
# Select CUDA device index
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-large"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

from peft import LoraConfig, get_peft_model, TaskType


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM"
)


model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

import torch
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils import resample

import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

from sklearn.model_selection import train_test_split

from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

trainable params: 4718592 || all params: 787868672 || trainable%: 0.5989059049678777


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

peft_model_id = "kenchenxingyu/flan-large-lora-stance-human7"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [ ]:
origin_data = pd.read_pickle('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/truth_seeker_NER_emotion_stance.h5',compression='xz')
origin_data['source_text'] = "Classify stance towards the target:" + origin_data['target'].apply(lambda x: str(x)) + " in the text: "  + origin_data['tweet']
origin_data

,author,statement,BinaryNumTarget,tweet,timestamp,PARENT_ID,TWEET_ID,NER,target_type,target,target_filter,source_text,flan_emotion_pred,emotion
0,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,"Says Donald Trump has ""changed his mind"" on ab...",Fri Apr 01 00:28:51 +0000 2016,358,358,"(Donald Trump, PERSON)",PERSON,Donald Trump,True,Classify stance towards the target:Donald Trum...,4,neutral
1,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,@realDonaldTrump donald trump I so wanted to ...,Fri Apr 01 00:28:51 +0000 2016,358,45030,"(@realDonaldTrump donald trump , PERSON)",PERSON,@realDonaldTrump donald trump,True,Classify stance towards the target:@realDonald...,4,neutral
2,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Trump, PERSON)",PERSON,Trump,True,Classify stance towards the target:Trump in th...,4,neutral
3,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Heller, PERSON)",PERSON,Heller,True,Classify stance towards the target:Heller in t...,4,neutral
4,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Cruz, PERSON)",PERSON,Cruz,True,Classify stance towards the target:Cruz in the...,4,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293870,Sean Gorman,"Says President Barack Obama ""will not utter th...",1.0,Remember when Republicans hammered Obama becau...,Wed Sep 30 23:16:49 +0000 2020,330,39776,"(Trump, PERSON)",PERSON,Trump,True,Classify stance towards the target:Trump in th...,4,neutral
293871,Sue Owen,"""Texas has the highest rate of uninsured in th...",1.0,@JohnCornyn Texas has the highest rate of unin...,Wed Sep 30 23:44:29 +0000 2020,539,70635,"(Texas, GPE)",GPE,Texas,True,Classify stance towards the target:Texas in th...,4,neutral
293872,Jon Greenberg,Says President Barack Obama spied on my campai...,0.0,"Corrupt Joe Biden and Obama, who spied on my c...",Wed Sep 30 23:55:01 +0000 2020,988,125850,"(Joe Biden, PERSON)",PERSON,Joe Biden,True,Classify stance towards the target:Joe Biden i...,4,neutral
293873,Jon Greenberg,Says President Barack Obama spied on my campai...,0.0,"Corrupt Joe Biden and Obama, who spied on my c...",Wed Sep 30 23:55:01 +0000 2020,988,125850,"(Obama, PERSON)",PERSON,Obama,True,Classify stance towards the target:Obama in th...,4,neutral


In [ ]:
from IPython.display import clear_output

df = origin_data

samples_number = len(df)
progress_bar = tqdm(range(samples_number))
predictions_list = []


for i in range(samples_number):
    input_text = df['source_text'][i]

    if len(input_text.split()) < 1000:
        inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True,return_tensors="pt")
    else:
        inputs = tokenizer(input_text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

    # Move the input tensors to the GPU if they are not already there
    #inputs = {key: value.to('cuda') for key, value in inputs.items()}
    #print(inputs)

    outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'),
                             attention_mask=inputs['attention_mask'].to('cuda'),
                             max_length=10,
                             #num_beams=4,
                             #early_stopping=True
                             )
    del(inputs)
    torch.cuda.empty_cache()
    #gc.collect()

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions_list.append(prediction)
    #labels_list.append(test_df['label'][i])
    clear_output(wait=True)
    progress_bar.update(1)

origin_data['flan_stance'] = predictions_list

reverse_stance_mapping = {0:'AGAINST',
                  1:'NEUTRAL',
                  2:'FAVOR',
}

origin_data['stance'] = origin_data['flan_stance'].apply(lambda x: int(x)).map(reverse_stance_mapping)

origin_data.to_pickle('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/truth_seeker_NER_emotion_stance.h5',compression='xz')

100%|██████████| 293875/293875 [20:30:27<00:00,  4.02it/s]